# [LAB 13] 7. 선행,후행변수_검정
(Granger Causality Test, 그랜저 인과성 검정)

## 📘 #01. Granger Causality Test 개요
한 시계열 데이터가 다른 시계열 데이터를 예측하는 데 도움을 주는지 확인하는 방법

- "원인"을 밝히는 것이 아니라 **예측력의 선행 관계**를 검정하는 기법
- 예: 금리 변화가 아파트 가격 변화를 예측하는 데 도움이 되는가?

### 📝 [1] 핵심 개념
- 시간 순서가 있는 데이터(시계열) 필요
- **과거 값(past values)**을 이용해 미래 값을 예측
- **비교 구조:**
  - 자기 자신의 과거만 사용한 모델
  - 다른 변수의 과거까지 포함한 모델
  - 두 모델의 예측력이 유의하게 차이 나는지 확인
- 시간의 흐름이 핵심인 분석 기법

### 📝 [2] "인과"라는 표현의 의미
- 원인-결과를 **증명하는 것이 아님**
- 단지 "A의 과거 정보가 B를 예측하는 데 통계적으로 도움이 되는가?"
- 그래서 보통 다음과 같이 표현함
  - "A가 B를 그랜저 인과한다"
  - "A → B의 예측 선행 관계가 있다"

### 📝 [3] 사용 조건
- 시계열 데이터여야 함
- 일정한 시간 간격 유지 필요
- 데이터가 충분히 길어야 함
- 일반적으로 정상성(추세 제거 등) 확인 후 사용

### 📝 [4] 사용 예시
- 금리 → 주택가격
- 소비자심리지수 → 부동산 거래량
- 실업률 → 상업용 부동산 공실률
- 유동인구 → 매출액

## 📘 #02. 준비작업

### 📝 [1] 패키지 참조

In [ ]:
from hossam import *
from pandas import to_datetime, merge, DataFrame
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller, grangercausalitytests

### 📝 [2] 데이터셋 불러오기
데이터 로딩에 다소 시간이 걸린다.

In [ ]:
origin1 = load_data("apartment_transaction_prices")
origin1.head()

In [ ]:
origin2 = load_data("hhbl")
origin2.head()

### 📝 [3] 날짜 타입 변환

## 📘 #03. 아파트 평당단가와 가계대출의 관계

### 📝 [1] 거래데이터 → 분기 평균 단가 생성

In [ ]:
origin1['CNDT'] = to_datetime(origin1['CNDT'])
origin2['QTRT'] = to_datetime(origin2['QTRT'])
origin1['QTRT'] = origin1['CNDT'].dt.to_period('Q').dt.to_timestamp()
price_q = origin1.groupby('QTRT')['UPPP'].mean().reset_index()
price_q.head()

### 📝 [2] 병합

In [ ]:
merged_data = merge(price_q, origin2, on='QTRT', how='inner')
merged_data = merged_data.sort_values('QTRT').dropna()
merged_data.head()

### 📝 [3] 스케일링
필요하다면 스케일링 전 로그변환이 선행될 수 도 있음 (여기서는 생략)

In [ ]:
df = merged_data[['UPPP', 'HHBL']].copy()
scaler = StandardScaler()
sdf = DataFrame(scaler.fit_transform(df), columns=df.columns)
sdf.index = merged_data['QTRT']
sdf.head()

### 📝 [4] 차분

#### ✏ 정상성이 확보될 때 까지의 차수 계산 함수

In [ ]:
def get_diff_order(series, alpha=0.05):
    temp = series.copy()
    d = 0
    while True:
        result = adfuller(temp.dropna())
        if result[1] <= alpha:
            return d
        temp = temp.diff()
        d += 1

#### ✏ 함수 적용

In [ ]:
d_up = get_diff_order(sdf['UPPP'])
d_hh = get_diff_order(sdf['HHBL'])
print("UPPP 차수:", d_up)
print("HHBL 차수:", d_hh)

#### ✏ 동일 차수 선택

In [ ]:
common_d = max(d_up, d_hh)
print("공통 차수 적용:", common_d)

#### ✏ 동일 차수로 차분 적용

In [ ]:
stationary_df = sdf[['UPPP', 'HHBL']].copy()
for _ in range(common_d):
    stationary_df = stationary_df.diff()
stationary_df = stationary_df.dropna()
stationary_df.head()

### 📝 [5] Granger Causality Test

#### ✏ max_lag = 4 의 의미
- 최대 몇 시점 과거까지 확인할 것인가를 정하는 값
- Granger 검정은 여러 시차(lag)를 각각 따로 검정함
- max_lag = 4 → 1~4시차까지 모두 검사

#### ✏ 실무기준
| 데이터 주기 | 권장 max_lag |
|------------|-------------|
| 월간 | 6~12 |
| 분기 | 4~8 |
| 연간 | 2~4 |

In [ ]:
max_lag = 8  # 분기 데이터: 8로 확대
alpha = 0.05
granger_results = {}

# =========================================
# 1. 가계대출 변화 → 가격 변화 (y먼저)
# =========================================
test_forward = stationary_df[['UPPP', 'HHBL']].dropna()
min_p_forward = float('inf')
optimal_forward = None
f_stat_forward = None
result_forward = grangercausalitytests(
    test_forward,
    maxlag=max_lag,
    verbose=False
)
for lag, result in result_forward.items():
    p_value = result[0]['ssr_ftest'][1]
    f_stat = result[0]['ssr_ftest'][0]
    if p_value < min_p_forward:
        min_p_forward = p_value
        optimal_forward = lag
        f_stat_forward = f_stat

# =========================================
# 2. 가격 변화 → 가계대출 변화
# =========================================
test_reverse = stationary_df[['HHBL', 'UPPP']].dropna()
min_p_reverse = float('inf')
optimal_reverse = None
f_stat_reverse = None
result_reverse = grangercausalitytests(
    test_reverse,
    maxlag=max_lag,
    verbose=False
)
for lag, result in result_reverse.items():
    p_value = result[0]['ssr_ftest'][1]
    f_stat = result[0]['ssr_ftest'][0]
    if p_value < min_p_reverse:
        min_p_reverse = p_value
        optimal_reverse = lag
        f_stat_reverse = f_stat

# =========================================
# 3. 방향 판정
# =========================================
if min_p_forward < alpha and min_p_reverse >= alpha:
    category = f"선행 ({optimal_forward}step)"
elif min_p_reverse < alpha and min_p_forward >= alpha:
    category = f"후행 ({optimal_reverse}step)"
elif min_p_forward < alpha and min_p_reverse < alpha:
    category = f"양방향 (상호 영향)"
else:
    category = f"❌ 무관 (선행/후행 관계 없음)"

#### ✏ 💡 인사이트
- **양방향 인과성 성립**  
  - Forward p-value < 0.05 → 가계대출 → 평당단가 유의  
  - Reverse p-value < 0.05 → 평당단가 → 가계대출 유의  
  - 두 방향 모두 유의하므로 상호 영향 관계로 판정됨
- **최적 시차 구조**  
  - Forward 최적 lag → 가계대출 변화가 N분기 후 평당단가 변화와 연관  
  - Reverse 최적 lag → 평당단가 변화가 N분기 후 가계대출 변화와 연관  
  - 시차 크기가 다르므로 영향의 시간 지연이 비대칭임
- **검정 강도(F-통계량)**  
  - F값 비교로 어느 방향 설명력이 더 강한지 해석 가능
- **통계적 확실성**  
  - p-value가 0.05보다 작으면 단순 우연으로 볼 가능성이 낮음

#### ✏ 결과 정리

In [ ]:
# =========================================
# 4. 결과 정리
# =========================================
granger_results = {
    'Forward_P_Value': [f"{min_p_forward:.6f}"],
    'Forward_F_Stat': [f"{f_stat_forward:.4f}"],
    'Forward_Lag': [optimal_forward],
    'Reverse_P_Value': [f"{min_p_reverse:.6f}"],
    'Reverse_F_Stat': [f"{f_stat_reverse:.4f}"],
    'Reverse_Lag': [optimal_reverse],
    'Interpretation': [category]
}
granger_results_df = DataFrame(granger_results)
granger_results_df

## 📘 #04. Granger 결과가 회귀모형에 갖는 의미

### 📝 [1] 방향에 따른 결과 해석

| Granger 결과 | 통계적 의미 | 회귀모형 설계 의미 | 적용 전략 |
|-------------|------------|-------------------|----------|
| 선행 (X → UPPP) | X의 과거가 UPPP 예측 | ✔ 핵심 설명변수 후보 X.shift(k) 생성 | lag 생성 |
| 후행 (UPPP → X) | X는 가격에 반응 | ❌ 예측변수 부적절 | 사용하지 않음 |
| 양방향 | 상호 영향 | ⚠ 동시성 가능성 | Forward_Lag만 사용 (동시항 제외) |
| 무관 | 선행성 없음 | ❌ 단변량 예측력 낮음 | 제외 또는 ML 검증 |

### 📝 [2] 데이터셋 적용 방법
- **선행이면** → lag 생성
- **후행이면** → 사용하지 않음
- **양방향이면** → lag만 사용 (동시항 제외)

#### ✏ 예시 : 결과가 다음과 같다고 가정
| 변수 | Category | Lag |
|------|----------|-----|
| HHBL | 선행 | 4 |
| BIRR | 후행 | 2 |
| HPCS | 양방향 | 1(FL), 2(RL) |

### 📝 [2] 회귀모형 구성
아래 코드는 예시 입니다. 실습용이 아닙니다.

In [ ]:
import statsmodels.api as sm

df = stationary_df.copy()
# 예: HHBL 선행 4분기
df['HHBL_lag4'] = df['HHBL'].shift(4)
# 예: HPCS 양방향 1분기 → Forward_Lag만 사용
# df['HPCS_lag1'] = df['HPCS'].shift(1)
# BIRR 후행 → 생성 안 함
# 결측 제거
df_model = df.dropna()
df_model.head()

In [ ]:
X = df_model[['HHBL_lag4']]
X = sm.add_constant(X)
y = df_model['UPPP']
model = sm.OLS(y, X).fit()
print(model.summary())